Centralized server using Dagshub for experiment tracking and model registry

In [1]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Step 1: Create an imbalanced binary classification dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=2, n_redundant=8, 
                           weights=[0.9, 0.1], flip_y=0, random_state=42)

np.unique(y, return_counts=True)

(array([0, 1]), array([900, 100]))

In [3]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

Handle class imbalance

In [4]:
from imblearn.combine import SMOTETomek

smt = SMOTETomek(random_state=42)
X_train_res, y_train_res = smt.fit_resample(X_train, y_train)
np.unique(y_train_res, return_counts=True)

(array([0, 1]), array([619, 619]))

Track Experiments

In [5]:
models = [
    (
        "Logistic Regression", 
        {"C": 1, "solver": 'liblinear'},
        LogisticRegression(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "Random Forest", 
        {"n_estimators": 30, "max_depth": 3},
        RandomForestClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier With SMOTE",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train_res, y_train_res),
        (X_test, y_test)
    )
]

In [6]:
reports = []

for model_name, params, model, train_set, test_set in models:
    X_train = train_set[0]
    y_train = train_set[1]
    X_test = test_set[0]
    y_test = test_set[1]
    
    model.set_params(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)

In [7]:
import mlflow

In [8]:
# dagshub setup

import dagshub
dagshub.init(repo_owner='priya.vijayakumar1304', repo_name='mlops-with-mlflow-and-dagshub', mlflow=True)


❗❗❗ AUTHORIZATION REQUIRED ❗❗❗



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=8ffadf27-da6f-4e3b-aaae-01016752acfa&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=812c1b203567d00b802558a24a2f6e74364ed40d61f042c046ac31bbc229774d




Accessing as priya.vijayakumar1304

Initialized MLflow to track repo "priya.vijayakumar1304/mlops-with-mlflow-and-dagshub"

Repository priya.vijayakumar1304/mlops-with-mlflow-and-dagshub initialized!

In [9]:
# Initialize MLflow
mlflow.set_experiment("Anomaly Detection")
# mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_tracking_uri("https://dagshub.com/priya.vijayakumar1304/mlops-with-mlflow-and-dagshub.mlflow")
for i, element in enumerate(models):
    model_name = element[0]
    params = element[1]
    model = element[2]
    report = reports[i]
    
    with mlflow.start_run(run_name=model_name):        
        mlflow.log_params(params)
        mlflow.log_metrics({
            'accuracy': report['accuracy'],
            'recall_class_1': report['1']['recall'],
            'recall_class_0': report['0']['recall'],
            'f1_score_macro': report['macro avg']['f1-score']
        })  
        
        if "XGB" in model_name:
            mlflow.xgboost.log_model(model, "model")
        else:
            mlflow.sklearn.log_model(model, "model")  

2025/12/16 21:52:28 INFO mlflow.tracking.fluent: Experiment with name 'Anomaly Detection' does not exist. Creating a new experiment.
2025/12/16 21:52:32 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run Logistic Regression at: https://dagshub.com/priya.vijayakumar1304/mlops-with-mlflow-and-dagshub.mlflow/#/experiments/0/runs/96f2dc47ac92416dab005d7bb40dd0ba
🧪 View experiment at: https://dagshub.com/priya.vijayakumar1304/mlops-with-mlflow-and-dagshub.mlflow/#/experiments/0


2025/12/16 21:52:56 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run Random Forest at: https://dagshub.com/priya.vijayakumar1304/mlops-with-mlflow-and-dagshub.mlflow/#/experiments/0/runs/6ea9bcba4d3e42348023dd7c79015854
🧪 View experiment at: https://dagshub.com/priya.vijayakumar1304/mlops-with-mlflow-and-dagshub.mlflow/#/experiments/0


2025/12/16 21:53:13 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run XGBClassifier at: https://dagshub.com/priya.vijayakumar1304/mlops-with-mlflow-and-dagshub.mlflow/#/experiments/0/runs/cb2530db188347b8a1833a09cbcbfc53
🧪 View experiment at: https://dagshub.com/priya.vijayakumar1304/mlops-with-mlflow-and-dagshub.mlflow/#/experiments/0


2025/12/16 21:53:35 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run XGBClassifier With SMOTE at: https://dagshub.com/priya.vijayakumar1304/mlops-with-mlflow-and-dagshub.mlflow/#/experiments/0/runs/414c0f8cbbc243068d279c3afa04c011
🧪 View experiment at: https://dagshub.com/priya.vijayakumar1304/mlops-with-mlflow-and-dagshub.mlflow/#/experiments/0
